In [1]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import loadmat
import lmfit as lf

# functions I wrote previously for dealing with the suppression data
import suppression as s

You'll want the mat files and the struct: res.allData. Each row represents a trial. I never added a variable to label the headers so I describe them below. I bolded the variables you probably want to focus on. Hopefully you don't need to feed in the trials in the order they were presented to subjects but if so, you can figure it out with the other variables in the struct.


OS data:

 * Staircase number for a given test block (= file number)
 * Eye (1=weaker eye, 2= fellow eye)
 * Mask Orientation (0=parallel, 90=orthogonal)
 * Binocular condition (1= monocular, 2=dichoptic)
 * Mask Contrast (michelson)
 * Trial number for this staircase
 * Probe contrast recommended by staircase algorithm
 * Response Accuracy (1=correct, 0=incorrect)
 * Probe Contrast used (I don't remember it ever being different from #7 and was really just a sanity check)
 * Interval that probe was presented in (1 or 2)
 * File number (=test block)
 
SS data:

 * Staircase number for a given test block (= file number)
 * Eye (1=weaker eye, 2= fellow eye)
 * Mask Orientation (0=parallel, 90=orthogonal)
 * Binocular Condition (1= monocular, 2=dichoptic)
 * Trial number for this staircase
 * Contrast increment recommended by staircase algorithm
 * Response Accuracy (1=correct, 0=incorrect)
 * Mask Contrast  (michelson)
 * Probe location (1-4, let me know if you need to know which number represents which quadrant)
 * Response (1-4)
 * Probe contrast increment used (I don't remember it ever being different from #6 and was really just a sanity check)
 * File number (=test block)



### Orientation Suppression

In [2]:
os_data_file = "cleanData/ah_filteredData_OS.mat"
os_data_files = glob.glob("cleanData/*filteredData_OS.mat")

In [12]:
def load_individual_data(data_file, columns):
    """Function that loads individual psychophysics data stored in .mat files.
    
    data_file: path to .mat file
    columns: list of column names. must match number of columns in the .mat file"""
    subject_code = data_file.split('_filteredData')[0].split('/')[-1] # get subject name from filename
    mat = loadmat(data_file)
    mdata = mat['data']
    mdtype = mdata.dtype
    subj_df = pd.DataFrame(data=mdata, columns=columns)
    
    # Convert the following columns to ints if they exist in this data frame
    cols_to_intify = ["StaircaseNumber", "Eye", "Orientation", "Presentation", "TrialNumberStaircase",
                "ResponseAccuracy", "ProbeInterval", "ProbeLocation", "FileNumber"]
    # Furthermore, make these ones categorical too
    cols_to_categorize = ["Eye", "Presentation", "ResponseAccuracy"]
    for col in cols_to_intify:
        if col in subj_df.columns:
            subj_df[col] = subj_df[col].astype(int)
            if col in cols_to_categorize:
                subj_df[col] = pd.Categorical(subj_df[col])
                
    # Finally, round these ones to avoid floating point errors
    cols_to_round = ["ProbeContrastRecommended", "ProbeContrastUsed"]
    for col in cols_to_round:
        if col in cols_to_intify or col in cols_to_categorize:
            raise Error(f"Column {col} is listed as both integer and floating point!")
        subj_df[col] = np.round(subj_df[col], 2)

    # Make Subject column and put it first
    subj_df['Subject'] = subject_code
    subj_df = subj_df[['Subject', *columns]]
    return subj_df

def load_individual_os_data(data_file):
    """Load data for Orientation Suppression task which has 11 columns."""
    columns_os = ["StaircaseNumber", "Eye", "Orientation", "Presentation", "MaskContrast", "TrialNumberStaircase",
              "ProbeContrastRecommended", "ResponseAccuracy", "ProbeContrastUsed", "ProbeInterval", "FileNumber"]
    return load_individual_data(data_file, columns_os)

def load_individual_ss_data(data_file):
    """Load data for Surround Suppression task which has 12 columns."""
    columns_ss = ["StaircaseNumber", "Eye", "Orientation", "Presentation", "TrialNumberStaircase",
              "ProbeContrastRecommended", "ResponseAccuracy", "MaskContrast", "ProbeLocation",
                  "Response", "ProbeContrastUsed",  "FileNumber"]
    return load_individual_data(data_file, columns_ss)

In [7]:
os_df = load_individual_os_data(os_data_file)

In [9]:
# sort in order trials were administered, ie block, staricase, trial
os_df.sort_values(["FileNumber", "StaircaseNumber", "TrialNumberStaircase"], inplace=True)


In [10]:
# Since -1 and -2 represent baseline, these should both be true
print(np.all(os_df[os_df.Orientation==-1]['MaskContrast']==0))
print(np.all(os_df[os_df.Orientation==-2]['MaskContrast']==0))

True
True


In [8]:
os_df.head(n=20)

,Subject,StaircaseNumber,Eye,Orientation,Presentation,MaskContrast,TrialNumberStaircase,ProbeContrastRecommended,ResponseAccuracy,ProbeContrastUsed,ProbeInterval,FileNumber
0,ah,1,2,-1,1,0.0,0,99.00,1,99.00,1,1
1,ah,2,2,-1,1,0.0,0,99.00,1,99.00,2,1
2,ah,3,2,-1,1,0.0,0,99.00,1,99.00,1,1
3,ah,4,2,-1,1,0.0,0,99.00,1,99.00,2,1
4,ah,3,2,-1,1,0.0,1,15.00,1,15.00,2,1
5,ah,4,2,-1,1,0.0,1,15.00,1,15.00,2,1
6,ah,2,2,-1,1,0.0,1,15.00,1,15.00,2,1
7,ah,3,2,-1,1,0.0,2,15.00,1,15.00,2,1
8,ah,1,2,-1,1,0.0,1,15.00,1,15.00,2,1
9,ah,4,2,-1,1,0.0,2,15.00,1,15.00,2,1


### Grouping

In [11]:
gvars = ["Eye", "Orientation", "Presentation"]
gvars_mask = gvars + ["MaskContrast"]
gvars_masktarget = gvars_mask + ["ProbeContrastUsed"]
grouped = os_df.groupby(gvars_masktarget)
for (gv, gr) in grouped:
    print(gv, len(gr))

(1, -2, 1, 0.0, 1.1200000000000001) 7
(1, -2, 1, 0.0, 1.5) 24
(1, -2, 1, 0.0, 2.0) 31
(1, -2, 1, 0.0, 2.6699999999999999) 40
(1, -2, 1, 0.0, 3.5600000000000001) 33
(1, -2, 1, 0.0, 4.7400000000000002) 12
(1, -2, 1, 0.0, 6.3300000000000001) 8
(1, -2, 1, 0.0, 8.4399999999999995) 8
(1, -2, 1, 0.0, 11.25) 8
(1, -2, 1, 0.0, 15.0) 16
(1, -2, 1, 0.0, 99.0) 4
(1, -1, 1, 0.0, 1.1200000000000001) 11
(1, -1, 1, 0.0, 1.5) 30
(1, -1, 1, 0.0, 2.0) 48
(1, -1, 1, 0.0, 2.6699999999999999) 41
(1, -1, 1, 0.0, 3.5600000000000001) 17
(1, -1, 1, 0.0, 4.7400000000000002) 10
(1, -1, 1, 0.0, 6.3300000000000001) 8
(1, -1, 1, 0.0, 8.4399999999999995) 8
(1, -1, 1, 0.0, 11.25) 8
(1, -1, 1, 0.0, 15.0) 16
(1, -1, 1, 0.0, 99.0) 4
(1, 0, 1, 11.0, 0.81999999999999995) 2
(1, 0, 1, 11.0, 1.1000000000000001) 3
(1, 0, 1, 11.0, 1.47) 7
(1, 0, 1, 11.0, 1.96) 15
(1, 0, 1, 11.0, 2.6099999999999999) 19
(1, 0, 1, 11.0, 3.48) 26
(1, 0, 1, 11.0, 4.6399999999999997) 24
(1, 0, 1, 11.0, 6.1900000000000004) 32
(1, 0, 1, 11.0, 8.25) 26


(2, 0, 1, 71.0, 16.84) 9
(2, 0, 1, 71.0, 22.449999999999999) 10
(2, 0, 1, 71.0, 29.940000000000001) 8
(2, 0, 1, 71.0, 39.93) 8
(2, 0, 1, 71.0, 53.240000000000002) 9
(2, 0, 1, 71.0, 71.0) 18
(2, 0, 2, 11.0, 0.81999999999999995) 3
(2, 0, 2, 11.0, 1.1000000000000001) 10
(2, 0, 2, 11.0, 1.47) 19
(2, 0, 2, 11.0, 1.96) 36
(2, 0, 2, 11.0, 2.6099999999999999) 38
(2, 0, 2, 11.0, 3.48) 28
(2, 0, 2, 11.0, 4.6399999999999997) 20
(2, 0, 2, 11.0, 6.1900000000000004) 12
(2, 0, 2, 11.0, 8.25) 8
(2, 0, 2, 11.0, 11.0) 16
(2, 0, 2, 18.0, 1.01) 2
(2, 0, 2, 18.0, 1.3500000000000001) 14
(2, 0, 2, 18.0, 1.8) 19
(2, 0, 2, 18.0, 2.3999999999999999) 15
(2, 0, 2, 18.0, 3.2000000000000002) 31
(2, 0, 2, 18.0, 4.2699999999999996) 31
(2, 0, 2, 18.0, 5.6900000000000004) 26
(2, 0, 2, 18.0, 7.5899999999999999) 20
(2, 0, 2, 18.0, 10.119999999999999) 16
(2, 0, 2, 18.0, 13.5) 10
(2, 0, 2, 18.0, 18.0) 16
(2, 0, 2, 28.0, 0.66000000000000003) 2
(2, 0, 2, 28.0, 0.89000000000000001) 5
(2, 0, 2, 28.0, 1.1799999999999999) 4
(2, 

#### Develop a function that should be minimized to fit the model

Within each staircase, you have numerous trials at different MaskContrasts, and as outcomes you have ResponseAccuracy (0|1). In the paper they describe these as the number of trials (n) and correct responses (c) for each target (here, ProbeContrastUsed) and mask (MaskContrast) level.

Ultimately, they fit their parameters (p, q, z, sigma_int - but mine will be different since i'm using a different model) by minimizing a likelihood function of these parameters given (vectors) m, t, n, c which are the different mask/target conditions.

Questions:
 * what are the parameters of the two-stage model, and which of those are going to be fixed and which are going to be free to vary, ie fit?
 * What is this normal integral d' to percent correct (P(c)) transformation?
     * Phi(t) is the cdf of the normal distribution, tells you the probability that the normal variable is less than t
     * We actually dont use d-prime at all in the two-stage model (they use it for their noise model), so we can just stay in %-correct land
 * What is the proper equation for the likelihood of the parameters given m, t, n, and c?
 * What is the proper function for calculating percent correct from the model parameters?

In [ ]:
params = lf.Parameters()
params.add('w_m', value=1, min=0.0, vary=True)
params.add('w_d', value=0.0249, min=0.0, vary=True)
params.add('a', value=0, vary=False)
params.add('k', value=0.2, vary=False)
params.add('p', value=8, vary=False)
params.add('q', value=6.5, vary=False)
params.add('Z', value=.0084, vary=False)

In [ ]:
s.two_stage_fac_resp(params, [1], [0], [0], [1])

### Surround Suppression

In [ ]:
ss_data_file = "cleanData/ah_filteredData_SS.mat"

In [ ]:
ss_df = load_individual_ss_data(ss_data_file)

In [ ]:
ss_df[ss_df.Orientation==-1]